In [49]:
import pandas as pd
import math
import folium
import gurobipy as gp
from gurobipy import GRB

df_stores = pd.DataFrame({
    'StoreID':  ['Store1', 'Store2', 'Store3'],
    'Latitude': [19.432608, 20.659699, 25.686614],
    'Longitude':[-99.133209, -103.349609, -100.316113],
    'Demand':   [10, 10, 15]   # Example demands
})

# Example DC data (lat/long in Mexico)
data_dcs = {
    'DCID':     ['DC1', 'DC2'],
    'Latitude': [20.659699, 19.432608],
    'Longitude':[ -100.316113, -103.349609],
    'Capacity':  [20,        30]
}
df_dcs = pd.DataFrame(data_dcs)

df_stores

,StoreID,Latitude,Longitude,Demand
0,Store1,19.432608,-99.133209,10
1,Store2,20.659699,-103.349609,10
2,Store3,25.686614,-100.316113,15


In [50]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points 
    on the Earth (specified in decimal degrees).
    Returns distance in kilometers.
    """
    # convert decimal degrees to radians 
    R = 6371  # Radius of the Earth in km
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = (math.sin(d_lat / 2) ** 2 
         + math.cos(math.radians(lat1)) 
         * math.cos(math.radians(lat2)) 
         * math.sin(d_lon / 2) ** 2)
    c = 2 * math.asin(math.sqrt(a))
    distance = R * c
    return distance

# Precompute distances between each store and DC
# We'll keep a dictionary of {(storeID, DCID) : distance}
distances = {}
for _, store_row in df_stores.iterrows():
    for _, dc_row in df_dcs.iterrows():
        dist = haversine_distance(store_row['Latitude'], store_row['Longitude'],
                                  dc_row['Latitude'], dc_row['Longitude'])
        distances[(store_row['StoreID'], dc_row['DCID'])] = dist

In [51]:
# !pip install gurobipy  # Uncomment if you need to install Gurobi (requires license)
# Create a new Gurobi model
m = gp.Model("Store_DC_Assignment_Capacity")

stores = df_stores['StoreID'].tolist()
dcs = df_dcs['DCID'].tolist()

# Decision variables x[i,j] = 1 if store i is assigned to DC j, else 0
x = m.addVars(stores, dcs, vtype=GRB.BINARY, name="assign")

# Objective: minimize total (distance * demand)
m.setObjective(
    gp.quicksum(
        distances[(i, j)] * 
        df_stores.loc[df_stores['StoreID'] == i, 'Demand'].values[0] * 
        x[i, j]
        for i in stores
        for j in dcs
    ), 
    GRB.MINIMIZE
)

# Constraint 1: each store is assigned to exactly one DC
for i in stores:
    m.addConstr(
        gp.quicksum(x[i,j] for j in dcs) == 1, 
        name=f"assign_store_{i}"
    )

# Constraint 2: DC capacity >= sum of demands of assigned stores
for j in dcs:
    capacity_j = df_dcs.loc[df_dcs['DCID'] == j, 'Capacity'].values[0]
    m.addConstr(
        gp.quicksum(
            df_stores.loc[df_stores['StoreID'] == i, 'Demand'].values[0] * x[i, j]
            for i in stores
        ) <= capacity_j,
        name=f"capacity_DC_{j}"
    )

# Solve
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B83)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x4a5675f7
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 14170.2 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.417020381495e+04, best bound 1.417020381495e+04, gap 0.0000%


In [52]:
if m.status == GRB.OPTIMAL:
    print("Optimal solution found!")
elif m.status == GRB.INFEASIBLE:
    print("No feasible solution.")
else:
    print(f"Solver ended with status {m.status}")

Optimal solution found!


In [53]:
assignment = []
if m.status == GRB.OPTIMAL:
    for i in stores:
        for j in dcs:
            if x[i,j].X > 0.5:  # i.e. 1
                assignment.append((i, j))
                print(f"Store {i} is assigned to DC {j}")

Store Store1 is assigned to DC DC2
Store Store2 is assigned to DC DC2
Store Store3 is assigned to DC DC1


In [54]:
df_assignment = pd.DataFrame(assignment, columns=['StoreID', 'DCID'])
df_assignment = df_assignment.merge(df_stores, on='StoreID', how='left', suffixes=('','_store'))
df_assignment = df_assignment.merge(df_dcs, on='DCID', how='left', suffixes=('_store','_dc'))
df_assignment

,StoreID,DCID,Latitude_store,Longitude_store,Demand,Latitude_dc,Longitude_dc,Capacity
0,Store1,DC2,19.432608,-99.133209,10,19.432608,-103.349609,30
1,Store2,DC2,20.659699,-103.349609,10,19.432608,-103.349609,30
2,Store3,DC1,25.686614,-100.316113,15,20.659699,-100.316113,20


In [55]:
# Create a Folium map centered on Mexico
m_map = folium.Map(location=[23.5, -102], zoom_start=5)

# Mark DCs in black
for _, row in df_dcs.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"DC: {row['DCID']}",
        icon=folium.Icon(color='black', icon='warehouse', prefix='fa')
    ).add_to(m_map)

# We can distinguish DCs by color
colors = [
    'blue', 'red', 'green', 'purple', 'orange', 'darkred', 
    'lightred', 'beige', 'darkblue', 'darkgreen'
]
dc_color_map = {}
for idx, dc_id in enumerate(dcs):
    dc_color_map[dc_id] = colors[idx % len(colors)]

# Mark stores and draw lines to their assigned DC
for _, row in df_assignment.iterrows():
    assigned_dc = row['DCID']
    
    # Plot the store marker
    folium.Marker(
        location=[row['Latitude_store'], row['Longitude_store']],
        popup=f"Store: {row['StoreID']} -> {assigned_dc}",
        icon=folium.Icon(color=dc_color_map[assigned_dc], icon='shopping-cart', prefix='fa')
    ).add_to(m_map)
    
    # Draw a line from the store to its assigned DC
    folium.PolyLine(
        locations=[
            [row['Latitude_store'], row['Longitude_store']],
            [row['Latitude_dc'], row['Longitude_dc']]
        ],
        color=dc_color_map[assigned_dc],
        weight=2
    ).add_to(m_map)

# Save map to HTML (optional)
m_map.save("store_dc_assignment_map_with_lines.html")

# Display the map in a Jupyter environment
m_map